1. Понимание бизнеса

1.1 Цель
Предсказать выживание на Титанике
1.2 Описание
Катастрофа Титаника - одно из самых печально известных кораблекрушений в истории. 15 апреля 1912 года во время своего первого плавания «Титаник» затонул после столкновения с айсбергом, убив 1502 из 2224 пассажиров и членов экипажа. Эта сенсационная трагедия потрясла международное сообщество и привела к улучшению правил безопасности судов.
Одна из причин того, что кораблекрушение привело к такому большому количеству жертв, заключалась в том, что на корбале не хватало спасательных шлюпок для пассажиров и членов экипажа. Несмотря на то, что в выживании в этой катастрофе был элемент случая, некоторые группы людей имели больше шансов выжить, чем другие (такие как женщины, дети и представители высшего класса).
В этой задаче мы просим вас завершить анализ того, какие люди могут выжить. В частности, мы просим вас применить инструменты машинного обучения, чтобы предсказать, какие пассажиры пережили трагедию.